In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, AutoConfig
import shap

2022-03-10 11:57:00.741297: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-10 11:57:00.741323: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def load_pipeline(INPUT_PATH):
    model = AutoModelForSequenceClassification.from_pretrained(INPUT_PATH)
    tokenizer = AutoTokenizer.from_pretrained(INPUT_PATH)
    config = AutoConfig.from_pretrained(INPUT_PATH)
    p = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, return_all_scores=True)
    return p

In [3]:
INPUT_PATH = '../../models/title'

pipeline_title = load_pipeline(INPUT_PATH)

In [4]:
INPUT_PATH = '../../models/title_summary'

pipeline_title_summary = load_pipeline(INPUT_PATH)

In [5]:
# Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue.

# LABEL_0--> CONTROVERSY, LABEL_1 --> NO_CONTROVERSY

In [26]:
def try_input(pipeline, INPUT_TEXT, display=True):
    print(pipeline([INPUT_TEXT]))
    explainer = shap.Explainer(pipeline)
    shap_values = explainer([INPUT_TEXT])
    shap_values.output_names = ['CONTROVERSY', 'NO_CONTROVERSY']
    html_out = shap.plots.text(shap_values[0, :, 'CONTROVERSY'], display = display) 
    if not display:
        file = open("../../output/visuals/"+INPUT_TEXT[:10].replace(' ', '_').lower()+".html","w")
        file.write(html_out)
        file.close()
    #return shap_values
    

In [7]:
INPUT_TEXT = "Guerra de Ucrania: Basta ya de mentiras, medias verdades y desinformación" # -- for adding the summary

html_out = try_input(pipeline_title, INPUT_TEXT)

[[{'label': 'LABEL_0', 'score': 0.31266161799430847}, {'label': 'LABEL_1', 'score': 0.6873384118080139}]]


  0%|          | 0/240 [00:00<?, ?it/s]

Partition explainer: 2it [00:17, 17.06s/it]                                                                                                                                                              


In [8]:
INPUT_TEXT = "Guerra de Ucrania: Basta ya de mentiras, medias verdades y desinformación -- El engaño de hoy será el lamento de mañana. Por ello, no solo hay que describir lo que está sucediendo desde múltiples puntos de vista, sino que también es necesario intentar alcanzar las razones por las que acontece. No es sencillo y, seguramente, no lo consiga debido a las múltiples limitaciones que poseo, pero es lo que he intentado hacer. Este es el resultado en quince puntos. No es una lectura cómoda ni corta, pero creo que necesaria. " # -- for adding the summary

try_input(pipeline_title_summary, INPUT_TEXT)

[[{'label': 'LABEL_0', 'score': 0.7468140721321106}, {'label': 'LABEL_1', 'score': 0.25318586826324463}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [01:07, 67.84s/it]                                                                                                                                                              


.values =
array([[[ 5.91687672e-03, -5.91687895e-03],
        [ 5.91687672e-03, -5.91687895e-03],
        [ 2.48553520e-02, -2.48553496e-02],
        [ 2.48553520e-02, -2.48553496e-02],
        [ 1.76555086e-02, -1.76555052e-02],
        [ 4.82525403e-03, -4.82525465e-03],
        [ 4.82525403e-03, -4.82525465e-03],
        [ 4.82525403e-03, -4.82525465e-03],
        [ 4.82525403e-03, -4.82525465e-03],
        [ 4.82525403e-03, -4.82525465e-03],
        [ 7.64303158e-03, -7.64302847e-03],
        [ 1.49196691e-02, -1.49196715e-02],
        [ 1.49196691e-02, -1.49196715e-02],
        [ 2.25165260e-02, -2.25165256e-02],
        [ 2.25165260e-02, -2.25165256e-02],
        [ 8.64558970e-03, -8.64558620e-03],
        [ 8.64558970e-03, -8.64558620e-03],
        [ 9.26241023e-03, -9.26240813e-03],
        [ 9.26241023e-03, -9.26240813e-03],
        [-2.80992860e-03,  2.80992797e-03],
        [-2.80992860e-03,  2.80992797e-03],
        [-2.80992860e-03,  2.80992797e-03],
        [-2.80992860e-

In [9]:
# TRY THE FIRST 4 INSTANCES IN WHICH T_S DOES IT RIGHT

In [10]:
import json

analysis_data = []
with open('analysis_data.json', 'r') as file:
    for line in file.readlines():
        d = json.loads(line)
        analysis_data.append(d)

In [11]:
for i,data in enumerate(analysis_data):
    if data['controversy'] and data['answer_summary'] == 'CONTROVERSY':
        print(i)

1
23
36
53
122
128
134
142
194
204
215
223
228
243
249
278
283
295
356
369
375
386
387
399
413
415
433
435
438
452
474
487
491
505
512
528
533
536
539
558
578
584
613
624
632
633
639
647
655
670
692
696
723
751
757
766
779
787
803
824
827
845
859
862
890
911
916
917
920
928
931
940
952
953
963
977
985
990
1041
1052
1071
1072
1083
1116
1122
1125
1147
1159
1164
1166
1168
1175
1186
1202
1205
1212
1240
1244
1257
1269
1279
1304
1313
1344
1362
1365
1383
1384


### Ejemplo 1

In [12]:
data = analysis_data[1]
print(data['answer_title'], data['answer_summary'])

CONTROVERSY CONTROVERSY


In [25]:
try_input(pipeline_title, "Los vecinos de Chamartín rodean y plan tan cara a los ultras de las cacerolas")

[[{'label': 'LABEL_0', 'score': 0.9431151151657104}, {'label': 'LABEL_1', 'score': 0.05688490346074104}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [00:27, 27.60s/it]                                                                                                                                                              


.values =
array([[[ 0.00000000e+00,  0.00000000e+00],
        [ 2.63777375e-03, -2.63774348e-03],
        [ 6.77257776e-06, -6.77583739e-06],
        [ 3.70509177e-03, -3.70510481e-03],
        [ 1.04331598e-02, -1.04331544e-02],
        [ 6.00937754e-03, -6.00939221e-03],
        [ 3.51931900e-03, -3.51931457e-03],
        [ 1.08266994e-02, -1.08266992e-02],
        [-4.36633080e-03,  4.36633732e-03],
        [-2.57439166e-03,  2.57439911e-03],
        [-4.86550108e-03,  4.86551551e-03],
        [ 1.14096589e-02, -1.14096631e-02],
        [ 1.90352090e-02, -1.90352100e-02],
        [-1.59155950e-03,  1.59155542e-03],
        [ 7.21539557e-03, -7.21538579e-03],
        [-3.40823457e-03,  3.40823503e-03],
        [-1.15982816e-03,  1.15983048e-03],
        [ 0.00000000e+00,  0.00000000e+00]]])

.base_values =
array([[0.8862825 , 0.11371747]])

.data =
(array(['', 'Los ', 'vecinos ', 'de ', 'Chamartín ', 'rodean ', 'y ',
       'plan ', 'tan ', 'cara ', 'a ', 'los ', 'ultras ', 'de ', 'l

In [14]:
try_input(pipeline_title_summary, data['title']+' -- '+data['content'])

[[{'label': 'LABEL_0', 'score': 0.7714641690254211}, {'label': 'LABEL_1', 'score': 0.22853587567806244}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [01:33, 93.28s/it]                                                                                                                                                              


.values =
array([[[ 6.31663818e-03, -6.31663734e-03],
        [ 6.31663818e-03, -6.31663734e-03],
        [ 1.39046168e-02, -1.39046160e-02],
        [ 1.39046168e-02, -1.39046160e-02],
        [ 2.95289021e-02, -2.95288969e-02],
        [ 2.06093236e-02, -2.06093197e-02],
        [ 8.42011388e-03, -8.42011098e-03],
        [-2.76619738e-04,  2.76617769e-04],
        [-2.76619738e-04,  2.76617769e-04],
        [ 3.92743835e-03, -3.92744218e-03],
        [ 4.50782182e-03, -4.50782378e-03],
        [ 6.59976316e-03, -6.59976328e-03],
        [ 6.59976316e-03, -6.59976328e-03],
        [ 6.37810001e-03, -6.37809966e-03],
        [ 6.37810001e-03, -6.37809966e-03],
        [ 1.30222418e-03, -1.30222500e-03],
        [ 1.30222418e-03, -1.30222500e-03],
        [ 1.30222418e-03, -1.30222500e-03],
        [ 1.20385286e-03, -1.20385354e-03],
        [ 1.20385286e-03, -1.20385354e-03],
        [ 1.20385286e-03, -1.20385354e-03],
        [ 1.20385286e-03, -1.20385354e-03],
        [ 2.74455749e-

In [15]:
data2 = analysis_data[23]
print(data2['answer_title'], data2['answer_summary'])

CONTROVERSY CONTROVERSY


In [16]:
try_input(pipeline_title, data2['title'], display = True)

[[{'label': 'LABEL_0', 'score': 0.52818363904953}, {'label': 'LABEL_1', 'score': 0.47181642055511475}]]


  0%|          | 0/210 [00:00<?, ?it/s]

Partition explainer: 2it [00:14, 14.16s/it]                                                                                                                                                              


.values =
array([[[ 0.        ,  0.        ],
        [-0.00585068,  0.0058507 ],
        [-0.01874664,  0.01874664],
        [ 0.00556283, -0.00556284],
        [-0.05133861,  0.05133859],
        [-0.03250001,  0.03250002],
        [-0.04703888,  0.04703887],
        [-0.06822098,  0.06822096],
        [-0.01856552,  0.01856553],
        [-0.0263256 ,  0.02632561],
        [-0.06416363,  0.06416366],
        [-0.04294287,  0.04294288],
        [-0.00094629,  0.0009463 ],
        [-0.00047802,  0.00047802],
        [ 0.        ,  0.        ]]])

.base_values =
array([[0.89973855, 0.10026146]])

.data =
(array(['', 'Quién ', 'manda ', 'en ', 'Castilla ', 'y ', 'León', ': ',
       'lo ', 'que ', 'nunca ', 'te ', 'han ', 'contado', ''],
      dtype=object),)

In [17]:
try_input(pipeline_title_summary, data2['title']+' -- '+data2['content'])

[[{'label': 'LABEL_0', 'score': 0.5583215355873108}, {'label': 'LABEL_1', 'score': 0.4416784942150116}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [01:27, 87.19s/it]                                                                                                                                                              


.values =
array([[[ 1.31092849e-03, -1.31092616e-03],
        [ 1.31092849e-03, -1.31092616e-03],
        [ 1.31092849e-03, -1.31092616e-03],
        [ 1.31092849e-03, -1.31092616e-03],
        [ 5.03829448e-04, -5.03828051e-04],
        [ 5.03829448e-04, -5.03828051e-04],
        [ 5.03829448e-04, -5.03828051e-04],
        [ 5.03829448e-04, -5.03828051e-04],
        [ 9.10300529e-04, -9.10300063e-04],
        [ 9.10300529e-04, -9.10300063e-04],
        [ 9.10300529e-04, -9.10300063e-04],
        [ 9.10300529e-04, -9.10300063e-04],
        [ 9.80238197e-04, -9.80236800e-04],
        [ 9.80238197e-04, -9.80236800e-04],
        [ 9.80238197e-04, -9.80236800e-04],
        [ 9.80238197e-04, -9.80236800e-04],
        [-4.04277956e-03,  4.04278003e-03],
        [-4.04277956e-03,  4.04278003e-03],
        [-3.67171178e-03,  3.67171224e-03],
        [-3.67171178e-03,  3.67171224e-03],
        [-3.09092055e-03,  3.09092086e-03],
        [-3.09092055e-03,  3.09092086e-03],
        [-3.09092055e-

In [18]:
data3 = analysis_data[36]
print(data3['answer_title'], data3['answer_summary'])

CONTROVERSY CONTROVERSY


In [19]:
try_input(pipeline_title, data3['title'], display=True)

[[{'label': 'LABEL_0', 'score': 0.5360153913497925}, {'label': 'LABEL_1', 'score': 0.4639846086502075}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [00:24, 24.11s/it]                                                                                                                                                              


.values =
array([[[ 0.        ,  0.        ],
        [-0.0174107 ,  0.01741071],
        [-0.00529983,  0.00529981],
        [-0.04176277,  0.04176274],
        [-0.03085005,  0.03085006],
        [-0.03059183,  0.03059184],
        [-0.01592675,  0.01592675],
        [-0.03362832,  0.03362834],
        [ 0.01109285, -0.01109282],
        [-0.01075527,  0.01075524],
        [-0.04483709,  0.04483709],
        [-0.06375466,  0.06375467],
        [-0.00567109,  0.00567108],
        [-0.03870877,  0.03870877],
        [-0.00714453,  0.00714452],
        [-0.02167913,  0.02167915],
        [ 0.        ,  0.        ]]])

.base_values =
array([[0.89294332, 0.10705667]])

.data =
(array(['', '"', 'Le ', 'voy ', 'a ', 'dar ', 'un ', 'dato', ', ',
       'señor ', 'García ', 'Egea', '..." ', '(', 'Twitter', ')', ''],
      dtype=object),)

In [20]:
try_input(pipeline_title_summary, data3['title']+' -- '+data3['content'])

[[{'label': 'LABEL_0', 'score': 0.6585212349891663}, {'label': 'LABEL_1', 'score': 0.34147876501083374}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [00:42, 42.83s/it]                                                                                                                                                              


.values =
array([[[-0.00134998,  0.00134999],
        [-0.00134998,  0.00134999],
        [-0.00147746,  0.00147746],
        [-0.00147746,  0.00147746],
        [ 0.00012711, -0.0001271 ],
        [ 0.00012711, -0.0001271 ],
        [ 0.00012711, -0.0001271 ],
        [ 0.00012711, -0.0001271 ],
        [ 0.00012711, -0.0001271 ],
        [ 0.00228363, -0.00228363],
        [ 0.00228363, -0.00228363],
        [ 0.00271884, -0.00271884],
        [ 0.00271884, -0.00271884],
        [ 0.0020136 , -0.0020136 ],
        [ 0.00292561, -0.00292562],
        [ 0.00292561, -0.00292562],
        [ 0.00120041, -0.00120041],
        [ 0.00120041, -0.00120041],
        [ 0.00283234, -0.00283234],
        [ 0.00283234, -0.00283234],
        [ 0.00951726, -0.00951726],
        [ 0.00841808, -0.00841809],
        [ 0.00318681, -0.00318681],
        [ 0.00318681, -0.00318681],
        [ 0.00053275, -0.00053275],
        [ 0.00053275, -0.00053275],
        [ 0.00053275, -0.00053275],
        [ 0.000532

In [21]:
data4 = analysis_data[53]
print(data4['answer_title'], data4['answer_summary'])

NO_CONTROVERSY CONTROVERSY


In [22]:
try_input(pipeline_title, data4['title'])

[[{'label': 'LABEL_0', 'score': 0.3481847047805786}, {'label': 'LABEL_1', 'score': 0.6518152356147766}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [00:26, 26.58s/it]                                                                                                                                                              


.values =
array([[[ 0.00041031, -0.0004103 ],
        [-0.01126303,  0.01126304],
        [-0.03223497,  0.03223496],
        [-0.05545249,  0.0554525 ],
        [-0.07137303,  0.07137302],
        [-0.03046651,  0.03046652],
        [-0.03001971,  0.03001971],
        [-0.03150787,  0.03150786],
        [-0.04733824,  0.04733823],
        [ 0.03383873, -0.03383873],
        [-0.04198854,  0.04198852],
        [ 0.00927398, -0.009274  ],
        [ 0.00199199, -0.00199198],
        [-0.06196968,  0.06196965],
        [-0.11557717,  0.11557716],
        [-0.09750209,  0.0975021 ],
        [-0.03715999,  0.03715999],
        [ 0.08806797, -0.088068  ],
        [ 0.        ,  0.        ]]])

.base_values =
array([[0.87845504, 0.12154497]])

.data =
(array(['', 'Las ', '"', 'famil', 'ias ', 'g', 'eno', 'ves', 'as', '": ',
       'nuevas ', 'parejas ', 'descu', 'biertas ', 'en ', 'el ', 'PP ',
       'madrileño', ''], dtype=object),)

In [23]:
try_input(pipeline_title_summary, data4['title']+' -- '+data4['content'])

[[{'label': 'LABEL_0', 'score': 0.4968467056751251}, {'label': 'LABEL_1', 'score': 0.5031533241271973}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [01:07, 67.21s/it]                                                                                                                                                              


.values =
array([[[-9.97746363e-04,  9.97746612e-04],
        [-9.97746363e-04,  9.97746612e-04],
        [-9.97746363e-04,  9.97746612e-04],
        [-1.53826891e-03,  1.53826900e-03],
        [-1.53826891e-03,  1.53826900e-03],
        [-9.89535413e-04,  9.89536178e-04],
        [-9.89535413e-04,  9.89536178e-04],
        [-9.89535413e-04,  9.89536178e-04],
        [-9.89535413e-04,  9.89536178e-04],
        [-2.79948248e-04,  2.79948160e-04],
        [-2.79948248e-04,  2.79948160e-04],
        [-2.79948248e-04,  2.79948160e-04],
        [-5.53608406e-04,  5.53607102e-04],
        [-5.53608406e-04,  5.53607102e-04],
        [ 4.04354651e-04, -4.04355023e-04],
        [ 1.32977078e-03, -1.32977115e-03],
        [ 1.19032479e-02, -1.19032434e-02],
        [ 1.19032479e-02, -1.19032434e-02],
        [ 4.16675683e-03, -4.16675465e-03],
        [ 4.16675683e-03, -4.16675465e-03],
        [ 7.46555937e-05, -7.46475843e-05],
        [ 7.46555937e-05, -7.46475843e-05],
        [-4.82079182e-